In [7]:
import sys
sys.path.append('../code')
from resnet import *
from funcs import *
from cifar_very_tiny import *
from cifar_dataset import *    
import torch as t 
import numpy as np
from numpy import polyfit
from numpy import polyval
import tqdm
import matplotlib.pylab as plt
import matplotlib.cm as cm
import json
import hyperparams
from importlib import reload
from scipy.interpolate import interp1d
from PIL import Image
%matplotlib inline
plt.rcParams['figure.figsize']=(12,9)
plt.rcParams['font.size']= 20

In [8]:
# добавил в загрузку валидационную выборку
# обрати внимание, maxsize --- это размер совокпного обучения и валидации
# поэтому размер обучающей выборки совпадает с тем, что было до этого

train_loader_no_augumentation, test_loader = cifar10_loader(batch_size=128, split_train_val=False,
                                                                             maxsize=128*100, use_aug=False)


Files already downloaded and verified
Files already downloaded and verified


In [9]:
epoch_num = 50
# epoch_num = 50

run_num = 1 # количество запусков эксперимента

# версия нужна, чтобы различать старые и новые результаты экспериментов. 
# менять нужно каждый раз, когда есть хотя бы незначительные изменения в эксперименте
experiment_version = '2'

validate_every_epoch = 5 

# train_splines_every_epoch = 5 # каждые 5 эпох отслеживать траекторию гиперпараметров
# train_splines_every_epoch = 2
# train_splines_every_epoch = 3
train_splines_every_epoch = 10

# размер мини-эпохи в батчах, за которую у нас производится либо обучение спайлов, либо их использование
mini_epoch_size = 10

start_beta = 0.5
start_temp  = 1.0

In [10]:
student = Cifar_Very_Tiny(10).cpu()
teacher = ResNet18(10).cpu() 
#scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=10, gamma=0.5)   
mu_feat = nn.Linear(512, 64).cpu()
log_sigma_feat = torch.nn.Parameter(torch.zeros(1).cpu())
optim = torch.optim.Adam(list(student.parameters()) + list(mu_feat.parameters()) + [log_sigma_feat])    

crit = nn.CrossEntropyLoss()

for e in range(25):
    tq = tqdm.tqdm(train_loader_no_augumentation)
    losses = []
            
    for batch_id, (x,y) in enumerate(tq):
        x = x.to(device)
        y = y.to(device)
        optim.zero_grad()
        student_feat, student_logits = student.get_features(x, [3,4])
        teacher_feat, teacher_logits = teacher.get_features(x, [3,4])
        class_loss = crit(student_logits, y)
        sigma2 = torch.log(1+torch.exp(log_sigma_feat))
        feat_loss = ((mu_feat(teacher_feat) - student_feat)**2).sum(1).mean()/sigma2 + torch.log(sigma2)*np.prod(teacher_feat.shape[1:])
        loss = class_loss * 0.9# + feat_loss * 0.1
        
        loss.backward()
        optim.step()
        losses.append(loss.detach().cpu().numpy())
        tq.set_description('current loss:{}'.format(np.mean(losses[-10:])))      
    student.eval()
    ac = float(accuracy(student, test_loader))
    print(e, ac)
    student.train()
    with open('../logs/acc_naive.txt','a') as out:
        out.write('{}: {}\n'.format(e, ac))


current loss:1.475279450416565:  96%|█████████▌| 96/100 [04:03<00:09,  2.44s/it]


current loss:1.4611507654190063: 100%|██████████| 100/100 [04:13<00:00,  2.53s/it]

  0%|          | 0/100 [00:00<?, ?it/s]

0 0.40709999203681946



current loss:1.2679882049560547:  96%|█████████▌| 96/100 [03:54<00:09,  2.49s/it]


current loss:1.2646390199661255: 100%|██████████| 100/100 [04:04<00:00,  2.45s/it]

  0%|          | 0/100 [00:00<?, ?it/s]

...

current loss:0.6903120279312134: 100%|██████████| 100/100 [04:02<00:00,  2.42s/it]

  0%|          | 0/100 [00:00<?, ?it/s]

23 0.5565000176429749



current loss:0.6907873749732971:  96%|█████████▌| 96/100 [03:46<00:10,  2.64s/it]


current loss:0.6819923520088196: 100%|██████████| 100/100 [03:57<00:00,  2.37s/it]


24 0.5547000169754028


In [12]:
!wget "https://github.com/passalis/pkth/blob/master/cifar10/models/resnet18_cifar10.model?raw=true"

--2022-03-23 13:06:07--  https://github.com/passalis/pkth/blob/master/cifar10/models/resnet18_cifar10.model?raw=true!wget
Распознаётся github.com (github.com)… 140.82.121.4
Подключение к github.com (github.com)|140.82.121.4|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://github.com/passalis/pkth/raw/master/cifar10/models/resnet18_cifar10.model [переход]
--2022-03-23 13:06:08--  https://github.com/passalis/pkth/raw/master/cifar10/models/resnet18_cifar10.model
Повторное использование соединения с github.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://raw.githubusercontent.com/passalis/pkth/master/cifar10/models/resnet18_cifar10.model [переход]
--2022-03-23 13:06:08--  https://raw.githubusercontent.com/passalis/pkth/master/cifar10/models/resnet18_cifar10.model
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Подключение к raw.gith

In [14]:
crit = nn.CrossEntropyLoss()
for lam1 in [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 0.1, 0.5]:
    student = Cifar_Very_Tiny(10).cpu()
    teacher = ResNet18(10).cpu() 
    teacher.load_state_dict(torch.load('resnet18_cifar10.model?raw=true', map_location=torch.device('cpu')))
    #scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=10, gamma=0.5)   
    mu_feat = nn.Linear(512, 64).cpu()
    log_sigma_feat = torch.nn.Parameter(torch.zeros(1).cpu())

    #mu_logit = nn.Linear(10, 10).cpu()
    #log_sigma_logit = torch.nn.Parameter(torch.zeros(1).cpu())


    optim = torch.optim.Adam(list(student.parameters()) + list(mu_feat.parameters()) + [log_sigma_feat])    


    for e in range(25):
        tq = tqdm.tqdm(train_loader_no_augumentation)
        losses = []

        for batch_id, (x,y) in enumerate(tq):
            x = x.to(device)
            y = y.to(device)
            optim.zero_grad()
            student_feat, student_logits = student.get_features(x, [3,4])
            teacher_feat, teacher_logits = teacher.get_features(x, [3,4])
            class_loss = crit(student_logits, y)
            sigma2 = torch.log(1+torch.exp(log_sigma_feat))
            feat_loss = ((mu_feat(teacher_feat) - student_feat)**2).sum(1).mean()/(2*sigma2) + 0.5*torch.log(sigma2)*np.prod(teacher_feat.shape[1:])
            #logit_loss =((mu_feat(teacher_feat) - student_feat)**2).sum(1).mean()/(2*sigma2) + 0.5*torch.log(sigma2)*np.prod(teacher_feat.shape[1:])
            #lam1 = 0.5
            loss = class_loss * (1.0-lam1) + feat_loss * lam1


            loss.backward()
            optim.step()
            losses.append(loss.detach().cpu().numpy())
            tq.set_description('current loss:{}'.format(np.mean(losses[-10:])))      
        #scheduler.step()
        student.eval()
        ac = float(accuracy(student, test_loader))
        print(e, ac)
        student.train()
        with open('../logs/acc_mi.txt','a') as out:
            out.write('{}: {}: {}\n'.format(lam1, e, ac))


current loss:1.629537582397461:  96%|█████████▌| 96/100 [11:44<00:29,  7.40s/it]


current loss:1.6203511953353882: 100%|██████████| 100/100 [12:14<00:00,  7.34s/it]

  0%|          | 0/100 [00:00<?, ?it/s]

0 0.400299996137619



current loss:1.439332365989685:  96%|█████████▌| 96/100 [11:43<00:29,  7.36s/it]


current loss:1.4445626735687256: 100%|██████████| 100/100 [12:12<00:00,  7.32s/it]

  0%|          | 0/100 [00:00<?, ?it/s]

...


current loss:0.8872398138046265:  96%|█████████▌| 96/100 [12:02<00:28,  7.07s/it]


current loss:0.8867232203483582: 100%|██████████| 100/100 [12:29<00:00,  7.50s/it]

  0%|          | 0/100 [00:00<?, ?it/s]

14 0.5522000193595886



current loss:0.8595542907714844:  46%|████▌     | 46/100 [05:26<06:23,  7.09s/it]


KeyboardInterrupt: 